# FizzBuzz in Qiskit

**Fizz buzz** is a group word game for children to teach them about division [1]. First person starts counting from $1$ then each next player takes turns adding another $1$ to previous number and saying the result except on occasion it should be replaced by
* *Fizz* if the result is divisible by $3$,
* *Buzz* if the result is divisible by $5$,
* *Fizz Buzz* if it happens simultaneously.

Those who make a mistake are out of the game which is continue round by round. As a result there is a word chain likes "One, Two, *Fizz*, Four, *Buzz*, *Fizz*, Seven, Eight" so on.

**FizzBuzz** is also a common programming task to test basic skills. In this case the sequence from $1$ to $100$ should be a program output and could be obtained in many ways. So there are a lot of its implementations in different programming languages. Let's try to realize a simplified version of FizzBuzz in Qiskit using Grover's Search Algorithm (GSA) for multiple solutions.

We create a FizzBuzz sequence from $0$ to $15$ due to some reasons. All the numbers have a four bit representation, two auxiliary bits is enough to carry a check condition that is simplified in the range, and there are two FizzBuzz opening and closing the chain.

GSA is an algorithm for unstructured search presented by Lov Grover in 1996 [2]. Instead of serial brute force it processes all data in parallel increasing a probability of desired outcomes in final distribution. That provides quantum supremacy becasue of quadratic speedup.

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, Aer, IBMQ, QuantumRegister, ClassicalRegister, execute
from qiskit.visualization import *

## Initialization

In [ ]:
def init(qc):
    qc.h(range(6))

## Quantum Fourier Transform

In [ ]:
def qft(qc):
    qc.h(5)
    qc.cp(np.pi/2, 4, 5)
    qc.h(4)
    qc.swap(4,5)

## Divisibility by 3

Since $3_{10}=11_2$ it is easy to check if a binary number satisfies the condition. The algorithm would be the same as for decimal number. We find the sums of its even and odd digits and compare them to each other. If they are equal or if their difference is multiple of $11_2$ then the binary number is divisible by $11_2$. The second statement is always false for 4-bit register so the only way is $$(c_0+c_2)-(c_1+c_3)=0$$

In [ ]:
def div_by_3(qc):
    qc.cp(np.pi/2, 0, 4)
    qc.cz(0, 5)
    qc.cp(np.pi/2, 2, 4)
    qc.cz(2, 5)
    qc.cp(-np.pi/2, 1, 4)
    qc.cz(1, 5)
    qc.cp(-np.pi/2, 3, 4)
    qc.cz(3, 5)

## Divisibility by 5

It looks a bit harder but very similar to the previous one. $3_{10}={0101}_2=11_4$ and each binary pair corresponds to a quaternary digit that is why we have to compare the sums of even and odd pairs of digits. The rest should be the same thus the condition reduces to $$c_1c_0-c_3c_2=0$$

In [ ]:
def div_by_5(qc):
    qc.cp(np.pi/2, 0, 4)
    qc.cz(0, 5)
    qc.cz(1, 4)
    qc.cp(-np.pi/2, 2, 4)
    qc.cz(2, 5)
    qc.cz(3, 4)

## Inverse Quantum Fourier Transform

In [ ]:
def iqft(qc):
    qc.h(4)
    qc.cp(-np.pi/2, 4, 5)
    qc.h(5)
    qc.swap(4, 5)

## Oracle

In [ ]:
def oracle(qc):
    qc.x(range(4, 6))
    qc.cz(4, 5)
    qc.x(range(4, 6))

## Diffuser

In [ ]:
fizz = QuantumRegister(4, 'fizz')
an_f = QuantumRegister(2, 'an_f')
cl_f = ClassicalRegister(6, 'cl_f')
fizz_circuit = QuantumCircuit(fizz, an_f, cl_f)
init(fizz_circuit)
#qft(fizz_circuit)
fizz_circuit.barrier()
div_by_3(fizz_circuit)
fizz_circuit.barrier()
iqft(fizz_circuit)
fizz_circuit.barrier()
oracle(fizz_circuit)
fizz_circuit.barrier()
fizz_circuit.h(range(6))
fizz_circuit.x(range(6))
fizz_circuit.mcp(np.pi, [0, 1, 2, 3, 4], 5)
fizz_circuit.x(range(6))
fizz_circuit.h(range(6))
fizz_circuit.barrier()
#fizz_circuit.measure(qr3, cr3)
fizz_circuit.draw()

In [ ]:
sim = Aer.get_backend('aer_simulator')
job3 = execute(qc3, sim, shots = 10000)
result3 = job3.result()
counts3 = result3.get_counts()
plot_histogram(counts3)

In [ ]:
print(counts3)

In [ ]:
job3 = execute(qc3, sim, shots = 10000)
result3 = job3.result()
counts3 = result3.get_counts()
plot_histogram(counts3)

In [ ]:
qr5 = QuantumRegister(6, 'q')
cr5 = ClassicalRegister(6, 'c')
qc5 = QuantumCircuit(qr5, cr5)
qc5.h(range(6))
qc5.barrier()
qc5.cp(np.pi/2, 0, 4)
qc5.cz(0, 5)
qc5.cz(1, 4)
qc5.cp(-np.pi/2, 2, 4)
qc5.cz(2, 5)
qc5.cz(3, 4)
qc5.barrier()
qc5.h(4)
qc5.cp(-np.pi/2, 4, 5)
qc5.h(5)
qc5.barrier()
qc5.x(4)
qc5.x(5)
qc5.cz(4, 5)
qc5.x(4)
qc5.x(5)
qc5.barrier()
qc5.h(range(6))
qc5.x(range(6))
qc5.mcp(np.pi, [0, 1, 2, 3, 4], 5)
qc5.x(range(6))
qc5.h(range(6))
qc5.barrier()
qc5.measure(qr5, cr5)
qc5.draw()

In [ ]:
job5 = execute(qc5, sim, shots = 10000)
result5 = job5.result()
counts5 = result5.get_counts()
plot_histogram(counts5)